Find the resisters needed for comparator hysteresis, power management systems a and b, when:
- $V_{cc} \neq V_{in}$ ($V_{cc}$ = constant)
- $V_{cc} = V_{in}$ (`_s` self powered)



In [45]:
import numpy as np

https://www.maximintegrated.com/en/design/technical-documents/app-notes/3/3616.html

<br><img src="img/comparator.png" alt="Drawing" style="width: 500px;"/>

#### A function for each step

In [46]:
# Step 1 : Calculate  R3

def calc_R3(Vref, ir3=0.2E-6, Vcc=5):
    "calculate resister R3 based on selected current ir3"
    
    #                                                _________
    # R3 = (Vref-Vout)/ir3, when OUT=low -->  Vref--|_________|--Vout=0
    #                                                   R3
    R3a = Vref / ir3 
    
    
    #                                                _________
    # R3 = (Vcc-Vref)/ir3, when OUT=high -->  Vref--|_________|--Vout=Vcc
    #                                                   R3
    R3b = (Vcc - Vref) / ir3 
    
    R3 = [r for r in [R3a, R3b] if r>0] # only +ve values
    
    return( round( min(R3), -4) )       # return minimum positive value of R3

In [47]:
# Step 2 : Decide hysteresis bandwidth

def Vhb(Vth, Vtl):
    return Vth - Vtl

###### <br><img src="img/Power_mgmt_derivation.png" alt="Drawing" style="width: 2000px;"/>

In [48]:
# Step 3 : Calculate R1

def calc_R1(R3, Vhb, Vcc=5):
    return round( R3 * Vhb / Vcc, -4 )

In [49]:
# Step 4 : Choose Vth
# Vth = 5.2

###### <br><img src="img/Power_mgmt_derivation2.png" alt="Drawing" style="width: 300px;"/>

In [50]:
# Step 5 : Calculate R2
def calc_R2(R1, R3, Vref, Vth):
    return round( Vref / ( ((Vth - Vref)/R1) - (Vref/R3) ), -4)


In [34]:
# Step 5 : Verify threshold voltages

def VTH(R1, R2, R3, Vref):
    return Vref * R1 * ((1/R1) + (1/R2) + (1/R3))

def VTL(R1, R3, Vth, Vcc=5):
    return Vth - (R1 * Vcc / R3)

#### Find resister values for each system config

In [58]:
# System A 
# a : comparator:LTC1540, 1.8V voltage regulator:TPS7A02
Vref_a = 1.18
Vth_a = 5.44 # 3.31#45
Vtl_a = 2.3

# # System B 
# # b : comparator:MAX9117, 1.8V voltage regulator:TPS7A02 (same regulator in both systems)
# Vref_b = 1.25
# Vth_b = 2.8
# Vtl_b = 1.6


In [59]:
# Step 1 : Calculate  R3 
R3a = calc_R3(Vref_a)
R3a_s = calc_R3(Vref_a, Vcc=Vtl_a)

# R3b = calc_R3(Vref_b)
# R3b_s = calc_R3(Vref_b, Vcc=Vtl_b)

print('{:.3e}'.format(R3a), 
      '{:.3e}'.format(R3a_s))
# print('{:.3e}'.format(R3b), 
#       '{:.3e}'.format(R3b_s))

5.900e+06 5.600e+06


In [60]:
# Step 2 : Decide hysteresis bandwidth

Vhb_a = Vhb(Vth_a, Vtl_a)
# Vhb_b = Vhb(Vth_b, Vtl_b)

In [61]:
# Step 3 : Calculate R1

R1a = calc_R1(R3a, Vhb_a)
R1a_s = calc_R1(R3a_s, Vhb_a, Vcc=Vtl_a)
# R1b = calc_R1(R3b, Vhb_b)
# R1b_s = calc_R1(R3b_s, Vhb_b, Vcc=Vtl_b)

print('{:.3e}'.format(R1a), 
      '{:.3e}'.format(R1a_s))
# print('{:.3e}'.format(R1b), 
#       '{:.3e}'.format(R1b_s))

3.710e+06 7.650e+06


In [62]:
# Step 5 : Calculate R2
R2a = calc_R2(R1a, R3a, Vref_a, Vth_a)
R2a_s = calc_R2(R1a_s, R3a_s, Vref_a, Vth_a)

# R2b = calc_R2(R1b, R3b, Vref_b, Vth_b)
# R2b_s = calc_R2(R1b_s, R3b_s, Vref_b, Vth_b)

print('{:.6e}'.format(R2a), 
      '{:.6e}'.format(R2a_s))

# print('{:.6e}'.format(R2b), 
#       '{:.6e}'.format(R2b_s))

1.240000e+06 3.410000e+06


In [63]:
# def calc_R2(R1, R3):
#     return 1 / (3.7/(Vref*R1) - 1/R1 - 1/R3)

# R2 = calc_R2(2.88, 6)

# print('{:.3e}'.format(R2))

In [64]:
VTHa = VTH(R1a, R2a, R3a, Vref_a)
VTLa = VTL(R1a, R3a, VTHa)

VTHa_s = VTH(R1a_s, R2a_s, R3a_s, Vref_a)
VTLa_s = VTL(R1a_s, R3a_s, VTHa_s, Vcc=Vtl_a)

# VTHb = VTH(R1b, R2b, R3b, Vref_b)
# VTLb = VTL(R1b, R3b, VTHb)

# VTHb_s = VTH(R1b_s, R2b_s, R3b_s, Vref_b)
# VTLb_s = VTL(R1b_s, R3b_s, VTHb_s, Vcc=Vtl_b)


print('Vtha =' + '{:.3e}'.format(VTHa) + '\n' + 'Vtla =' + '{:.3e}'.format(VTLa))
print()
print('Vtha_s =' + '{:.3e}'.format(VTHa_s) + '\n' + 'Vtla_s =' + '{:.3e}'.format(VTLa_s))
# print()
# print('Vthb =' + '{:.3e}'.format(VTHb) + '\n' + 'Vtlb =' + '{:.3e}'.format(VTLb))
# print()
# print('Vthb_s =' + '{:.3e}'.format(VTHb_s) + '\n' + 'Vtlb_s =' + '{:.3e}'.format(VTLb_s))

Vtha =5.452e+00
Vtla =2.308e+00

Vtha_s =5.439e+00
Vtla_s =2.297e+00


###### <br><img src="img/Power_mgmt_circuit_calculations-page-001.jpg" alt="Drawing" style="width: 2000px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-002.jpg" alt="Drawing" style="width: 2500px;"/>
<br><img src="img/Power_mgmt_circuit_calculations-page-002_.png" alt="Drawing" style="width: 2500px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-003.jpg" alt="Drawing" style="width: 2500px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-004.jpg" alt="Drawing" style="width: 2500px;"/>

